In [1]:
import pandas as pd

In [2]:
df = pd.concat(
    [
        pd.read_json("./responses-chunked-prefill.jsonl", lines=True).assign(
            chunked_prefill=True
        ),
        pd.read_json("./responses-no-chunked-prefill.jsonl", lines=True).assign(
            chunked_prefill=False
        ),
    ]
)

In [3]:
df["delay"] = pd.to_datetime(df["timestamp"]) - pd.to_datetime(df["run_start"])
df["delay"] = df["delay"].dt.total_seconds()

In [ ]:
df.groupby(["chunked_prefill", "run_id"])["delay"].last()

In [10]:
import duckdb


con = duckdb.connect()

In [11]:
df = con.sql("SELECT *, ROW_NUMBER() OVER (PARTITION BY run_id ORDER BY timestamp ASC) as run_idx FROM df").df()

In [ ]:
import seaborn as sns

sns.lineplot(
    df,
    x="run_idx",
    y="delay",
    hue="chunked_prefill",
    units="run_id",
    estimator=None
)